In [10]:
import pandas as pd

In [90]:
#####

# README: This code turns x-y coordinate files into x-y-z files that can be used by Ovito

#####

# Change the zip code here to your coordinate file and file paths

zip_code = '15090'

s1 = 'Coords Folder/'

s2 = '_building_coords.txt'

file = open(s1 + zip_code + s2)

coords = pd.read_csv(file, sep = ',', names = ['Class', 'x', 'y'])

file.close()

coords

,Class,x,y
0,R,-80.064898,40.598676
1,R,-80.057230,40.605936
2,R,-80.053911,40.605764
3,R,-80.039885,40.639494
4,R,-80.042522,40.592529
...,...,...,...
9403,R,-80.074073,40.615258
9404,R,-80.078338,40.596541
9405,G,-80.040046,40.583516
9406,R,-80.057666,40.652279


In [91]:
# Drop the class column from the frame

coords = coords.drop(['Class'], axis = 1)

# Create the normalized dataframe

normalized_data = pd.DataFrame()

measure = [[1, 0], [1, 0]]

index = 0

abs_max = 0

# Find the normalization factor for both x and y

for letter in ['x', 'y']:
    
    localMax = coords[letter].max()
    
    localMin = coords[letter].min()
    
    abs_local = max(abs(localMax), abs(localMin))
    
    if abs_local > abs_max:
        
        abs_max = abs_local

for letter in ['x', 'y']:
    
    normalized_data[letter] = coords[letter].divide(abs_max)
    
    mean = normalized_data[letter].mean()

    normalized_data[letter] = normalized_data[letter].subtract(mean)

    normalized_data[letter] = normalized_data[letter].multiply(10000)
    
    for scale in range(len(measure[index])): 
    
        num = measure[index][scale]
        
        num = num/10000 
    
        num = num + mean  
    
        num = num * abs_max  
        
        measure[index][scale] = num
        
    index += 1
    
scale = (((measure[0][0] - measure[0][1])**2 + (measure[1][0] - measure[1][1])**2)**(1/2))/2

# The multiplication factor here comes from the amount of feet in one degree of latitude. It's a rough measure

line = 'One degree of Ovito coords is about equal to ' + str(int(scale*364000)) + ' feet!'

normalized_data

,x,y
0,0.066413,-2.767449
1,1.023279,-1.861463
2,1.437517,-1.882953
3,3.187814,2.326351
4,2.858709,-3.534499
...,...,...
9403,-1.078517,-0.698187
9404,-1.610751,-3.033817
9405,3.167684,-4.659231
9406,0.968891,3.921743


In [85]:
normalized_data.insert(0, 'test', 'H')

normalized_data[''] = 0

normalized_data.loc[-1] = ['', '', '', '']

normalized_data.index = normalized_data.index + 1

normalized_data = normalized_data.sort_index()

r, c = normalized_data.shape

# Setting one of the 'z' values as 0.01 to prevent degeneracy (lol)

normalized_data.columns = ['1', '2', '3', '4']

normalized_data['4'][1] = 0.01

normalized_data.columns = [r - 1, '', '', '']

normalized_data

,9408,,,
0,,,,
1,H,0.0664128,-2.76745,0.01
2,H,1.02328,-1.86146,0
3,H,1.43752,-1.88295,0
4,H,3.18781,2.32635,0
...,...,...,...,...
9404,H,-1.07852,-0.698187,0
9405,H,-1.61075,-3.03382,0
9406,H,3.16768,-4.65923,0
9407,H,0.968891,3.92174,0


In [102]:
# Found here: https://stackoverflow.com/questions/41428539/data-frame-to-file-txt-python/41428596

normalized_data.to_csv(path_or_buf = s1 + zip_code + '_building_coords_reformatted.txt', index = False, header = True, sep = '\t', mode = 'a')

# Write the scale to a text file

file = open(s1 + zip_code + '_scale.txt', 'w+')

file.write(str(line))

file.close()